In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
del_col = []
for column in train_df.columns:
    if (train_df[column].dtype == float or train_df[column].dtype == int):
        pass
    else:
        del_col.append(column)
    if train_df[column].isnull().all():
        del_col.append(column)

print(del_col)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [76]:
clean_train_df = train_df.drop(del_col,axis=1)

In [77]:
clean_train_df = clean_train_df.drop(['LotFrontage','MasVnrArea','GarageYrBlt','Id','TotalBsmtSF','YrSold','MoSold','BsmtFullBath','BsmtHalfBath','TotRmsAbvGrd','MiscVal'],axis=1)

In [78]:
clean_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1460 non-null   int64
 1   LotArea        1460 non-null   int64
 2   OverallQual    1460 non-null   int64
 3   OverallCond    1460 non-null   int64
 4   YearBuilt      1460 non-null   int64
 5   YearRemodAdd   1460 non-null   int64
 6   BsmtFinSF1     1460 non-null   int64
 7   BsmtFinSF2     1460 non-null   int64
 8   BsmtUnfSF      1460 non-null   int64
 9   1stFlrSF       1460 non-null   int64
 10  2ndFlrSF       1460 non-null   int64
 11  LowQualFinSF   1460 non-null   int64
 12  GrLivArea      1460 non-null   int64
 13  FullBath       1460 non-null   int64
 14  HalfBath       1460 non-null   int64
 15  BedroomAbvGr   1460 non-null   int64
 16  KitchenAbvGr   1460 non-null   int64
 17  Fireplaces     1460 non-null   int64
 18  GarageCars     1460 non-null   int64
 19  Garage

In [79]:
clean_train_df.isna().sum()

MSSubClass       0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
Fireplaces       0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
SalePrice        0
dtype: int64

In [80]:
X_train =clean_train_df.drop(['SalePrice'],axis=1)
#.values.tolist()
Y_train =clean_train_df["SalePrice"]
#.values.tolist()

In [81]:
#X_train = X_train.transpose()
X_train.shape

(1460, 26)

In [117]:
print(X_train[:1])

   MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
0          60     8450            7            5       2003          2003   

   BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  1stFlrSF  ...  KitchenAbvGr  Fireplaces  \
0         706           0        150       856  ...             1           0   

   GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  \
0           2         548           0           61              0          0   

   ScreenPorch  PoolArea  
0            0         0  

[1 rows x 26 columns]


In [89]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))
print(normalizer.mean.numpy())
first = np.array(X_train[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

[[5.6897266e+01 1.0516828e+04 6.0993142e+00 5.5753412e+00 1.9712676e+03
  1.9848657e+03 4.4363971e+02 4.6549309e+01 5.6724030e+02 1.1626270e+03
  3.4699237e+02 5.8445206e+00 1.5154634e+03 1.5650687e+00 3.8287669e-01
  2.8664384e+00 1.0465754e+00 6.1301368e-01 1.7671232e+00 4.7298019e+02
  9.4244507e+01 4.6660267e+01 2.1954107e+01 3.4095886e+00 1.5060957e+01
  2.7589042e+00]]
First example: [[  60 8450    7    5 2003 2003  706    0  150  856  854    0 1710    2
     1    3    1    0    2  548    0   61    0    0    0    0]]

Normalized: [[ 0.07 -0.21  0.65 -0.52  1.05  0.88  0.58 -0.29 -0.94 -0.79  1.16 -0.12
   0.37  0.79  1.23  0.16 -0.21 -0.95  0.31  0.35 -0.75  0.22 -0.36 -0.12
  -0.27 -0.07]]


In [90]:
model = Sequential(
    [
        normalizer,
        tf.keras.Input(shape=(26,)),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(1, activation="linear")
    ],name= "model"

)
#, activation="linear"

In [91]:

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 26)                53        
_________________________________________________________________
input_10 (InputLayer)        multiple                  0         
_________________________________________________________________
dense_30 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_31 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 65        
Total params: 6,006
Trainable params: 5,953
Non-trainable params: 53
_________________________________________________________________


In [92]:
model.compile(
    loss="mean_absolute_error",
    #tf.keras.losses.mean_absolute_error(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

In [93]:
history = model.fit(
    X_train,Y_train,
    epochs=2000
)

Epoch 1/2000
46/46 [==============================] - 1s 2ms/step - loss: 180917.7656
Epoch 2/2000
46/46 [==============================] - 0s 2ms/step - loss: 180900.6406
Epoch 3/2000
46/46 [==============================] - 0s 2ms/step - loss: 180845.6875
Epoch 4/2000
46/46 [==============================] - 0s 2ms/step - loss: 180711.4531
Epoch 5/2000
46/46 [==============================] - 0s 2ms/step - loss: 180447.2812
Epoch 6/2000
46/46 [==============================] - 0s 2ms/step - loss: 180001.5938
Epoch 7/2000
46/46 [==============================] - 0s 2ms/step - loss: 179320.0938
Epoch 8/2000
46/46 [==============================] - 0s 2ms/step - loss: 178351.2188
Epoch 9/2000
46/46 [==============================] - 0s 2ms/step - loss: 177049.3750
Epoch 10/2000
46/46 [==============================] - 0s 2ms/step - loss: 175364.4375
Epoch 11/2000
46/46 [==============================] - 0s 2ms/step - loss: 173252.7500
Epoch 12/2000
46/46 [==============================]

In [94]:
model.evaluate(Y_train)

46/46 [==============================] - 0s 771us/step - loss: 0.0000e+00


0.0

In [97]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [98]:
history.history
plot_loss(history)
plt.show()

KeyError: 'val_loss'

In [99]:
test_df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [102]:
del_col = []
for column in test_df.columns:
    if (test_df[column].dtype == float or test_df[column].dtype == int):
        pass
    else:
        del_col.append(column)
    if test_df[column].isnull().all():
        del_col.append(column)

print(del_col)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [103]:
clean_test_df = test_df.drop(del_col,axis=1)

In [104]:
clean_test_df = clean_test_df.drop(['LotFrontage','MasVnrArea','GarageYrBlt','Id','TotalBsmtSF','YrSold','MoSold','BsmtFullBath','BsmtHalfBath','TotRmsAbvGrd','MiscVal'],axis=1)

In [105]:
clean_test_df.isna().sum()

MSSubClass       0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       1
BsmtFinSF2       1
BsmtUnfSF        1
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
Fireplaces       0
GarageCars       1
GarageArea       1
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
dtype: int64

In [108]:
clean_test_df = clean_test_df.fillna({'BsmtFinSF1': clean_test_df['BsmtFinSF1'].mean(), 
                                      'BsmtFinSF2':clean_test_df['BsmtFinSF2'].mean(),
                                      'BsmtUnfSF':clean_test_df['BsmtUnfSF'].mean(),
                                      #'TotalBsmtSF':clean_test_df['TotalBsmtSF'].mean(),
                                      #'BsmtFullBath':clean_test_df['BsmtFullBath'].mean(),
                                      #'BsmtHalfBath':clean_test_df['BsmtHalfBath'].mean(),
                                      'GarageCars':clean_test_df['GarageCars'].mean(),
                                      'GarageArea':clean_test_df['GarageArea'].mean()
                                     }
                                    )

In [109]:
clean_test_df.isna().sum()

MSSubClass       0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
Fireplaces       0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
dtype: int64

In [110]:
X_test = clean_test_df
#.values.tolist()
#Y_test =clean_test_df["SalePrice"]
#.values.tolist()

In [111]:
test_predictions = model.predict(X_test)

In [112]:
test_predictions

array([[124768.805],
       [160176.23 ],
       [187782.8  ],
       ...,
       [178092.61 ],
       [116408.445],
       [249145.3  ]], dtype=float32)

In [113]:
submission = test_df[["Id"]]
submission

,Id
0,1461
1,1462
2,1463
3,1464
4,1465
...,...
1454,2915
1455,2916
1456,2917
1457,2918


In [114]:
submission["SalePrice"] = test_predictions
submission

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Id,SalePrice
0,1461,124768.804688
1,1462,160176.234375
2,1463,187782.796875
3,1464,198747.328125
4,1465,182452.578125
...,...,...
1454,2915,95809.125000
1455,2916,85158.523438
1456,2917,178092.609375
1457,2918,116408.445312


In [116]:
submission.to_csv("submission.csv", index = None)